# Production Technology

The dataset contains `N = 441` firms observed over `T = 12` years, 1968-1979. There variables are: 
* `lcap`: Log of capital stock, $k_{it}$ 
* `lemp`: log of employment, $\ell_{it}$ 
* `ldsa`: log of deflated sales, $y_{it}$
* `year`: the calendar year of the observation, `year` $ = 1968, ..., 1979$, 
* `firmid`: anonymized indicator variable for the firm, $i = 1, ..., N$, with $N=441$. 

In [242]:
import numpy as np
from numpy import linalg as la
import pandas as pd
from io import StringIO
from tabulate import tabulate
from matplotlib import pyplot as plt
import seaborn as sns

# Import this weeks LinearModels .py file
import LinearModelsWeek3 as lm
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [243]:
dat = pd.read_csv('firms.csv')

In [244]:
#dat.sample(5)

In [245]:
#dat.year.unique()

# Descriptives

In [246]:
#dat.describe()
#dat

In [247]:
#dat[['lcap','lemp','ldsa']].hist();

In [248]:
#sns.scatterplot(x='lemp', y='ldsa', data=dat); 

# Converting data to numpy format 

In [249]:
#Beholder kun 3 år

#dat = dat[(dat['year'] >= 1968) & (dat['year'] <= 1970)]
#dat=dat.reset_index(drop=True)
#dat

In [250]:
dat.ldsa.values.shape

(5292,)

In [251]:
N = dat.firmid.unique().size
T = dat.year.unique().size
assert dat.shape[0] == N*T, f'Error: data is not a balanced panel'
print(f'Data has N={N} and T={T}')

Data has N=441 and T=12


Extract data from `pandas` to `numpy` arrays. 

In [252]:
y = dat.ldsa.values.reshape((N*T,1))

ones = np.ones((N*T,1))
l = dat.lemp.values.reshape((N*T,1))
k = dat.lcap.values.reshape((N*T,1))
x = np.hstack([l, k])

In [253]:
# Create transformation matrix
def demeaning_matrix(T):
    ones_T = np.ones(T)
    Q_T = np.identity(T) - (1 / T) * np.outer(ones_T, ones_T)
    return Q_T

# Print the matrix
Q_T = demeaning_matrix(T)
print(f'Demeaning matrix for T={T} \n', Q_T)

Demeaning matrix for T=12 
 [[ 0.91666667 -0.08333333 -0.08333333 -0.08333333 -0.08333333 -0.08333333
  -0.08333333 -0.08333333 -0.08333333 -0.08333333 -0.08333333 -0.08333333]
 [-0.08333333  0.91666667 -0.08333333 -0.08333333 -0.08333333 -0.08333333
  -0.08333333 -0.08333333 -0.08333333 -0.08333333 -0.08333333 -0.08333333]
 [-0.08333333 -0.08333333  0.91666667 -0.08333333 -0.08333333 -0.08333333
  -0.08333333 -0.08333333 -0.08333333 -0.08333333 -0.08333333 -0.08333333]
 [-0.08333333 -0.08333333 -0.08333333  0.91666667 -0.08333333 -0.08333333
  -0.08333333 -0.08333333 -0.08333333 -0.08333333 -0.08333333 -0.08333333]
 [-0.08333333 -0.08333333 -0.08333333 -0.08333333  0.91666667 -0.08333333
  -0.08333333 -0.08333333 -0.08333333 -0.08333333 -0.08333333 -0.08333333]
 [-0.08333333 -0.08333333 -0.08333333 -0.08333333 -0.08333333  0.91666667
  -0.08333333 -0.08333333 -0.08333333 -0.08333333 -0.08333333 -0.08333333]
 [-0.08333333 -0.08333333 -0.08333333 -0.08333333 -0.08333333 -0.08333333
   0

In [254]:
# Transform the data
y_demean = lm.perm(Q_T, y)
x_demean = lm.perm(Q_T, x)

#print x_demean
print(x_demean.shape)

(5292, 2)


In [255]:
#Laver labels
label_y = 'log Deflated sales'
label_x = [
    'log Employment', 
    'log Adjusted Capital Stock',  
]

In [256]:
# Create function to check rank of demeaned matrix, and return its eigenvalues.
def check_rank(x):
    print(f'Rank of demeaned x: {la.matrix_rank(x)}')
    lambdas, V = la.eig(x.T@x)
    np.set_printoptions(suppress=True)  # This is just to print nicely.
    print(f'Eigenvalues of within-transformed x: {lambdas.round(decimals=0)}')

# Check rank of demeaned x
check_rank(x_demean)

Rank of demeaned x: 2
Eigenvalues of within-transformed x: [ 58. 214.]


In [257]:
# Estimate using the demeaned variables, y_demean and x_demean
fe_result = lm.estimate(y_demean, x_demean, transform='fe', T=T)

# Print results
lm.print_table((label_y, label_x), fe_result, title='FE regression', floatfmt='.4f')
b_hat=fe_result['b_hat']

FE regression
Dependent variable: log Deflated sales

                              Beta      Se    t-values
--------------------------  ------  ------  ----------
log Employment              0.6942  0.0147     47.2447
log Adjusted Capital Stock  0.1546  0.0130     11.9311
R² = 0.477
σ² = 0.018


In [258]:
####################  WALD TEST #########################
#########################################################

In [259]:
R=np.array([1,1]).reshape(1,-1)
r=np.array([1]).reshape(1,-1)

In [260]:
varb=fe_result['cov']
varb.shape

(2, 2)

In [261]:
print(R)
print(R.shape)
print(r)
print(r.shape)
print(b_hat)
print(b_hat.shape)
print(varb)
print(varb.shape)

[[1 1]]
(1, 2)
[[1]]
(1, 1)
[[0.69422625]
 [0.15461952]]
(2, 1)
[[ 0.00021592 -0.00010743]
 [-0.00010743  0.00016794]]
(2, 2)


In [262]:
W=(R@b_hat-r).T@np.linalg.inv(R@varb@(R.T))@(R@b_hat-r)
W.shape

W

array([[135.18994663]])

In [263]:
degrees_of_freedom = R.shape[0]  # Degrees of freedom
p_value = 1 - chi2.cdf(W, degrees_of_freedom)

In [264]:
#################### TEST FOR FE.1 ######################
#########################################################

In [265]:
# Make function to calculate the serial correlation
def serial_corr(y, x, T):
    # Calculate the residuals
    b_hat = lm.est_ols(y, x)
    e = y - x@b_hat
    
    # Create a lag transformation matrix
    L_T = np.eye(T, k=-1)
    L_T = L_T[1:]

    # Lag residuals
    e_l = lm.perm(L_T, e)

    # Create a transformation matrix that removes the first observation of each individal
    I_T = np.eye(T, k=0)
    I_T = I_T[1:]
    
    # Remove first observation of each individual
    e = lm.perm(I_T, e)
    
    # Calculate the serial correlation
    return lm.estimate(e, e_l,T=T-1)

In [266]:
# Estimate serial correlation
corr_result = serial_corr(y_demean, x_demean, T)

# Print results
label_ye = 'OLS residual, e\u1d62\u209c'
label_e = ['e\u1d62\u209c\u208B\u2081']
lm.print_table(
    (label_ye, label_e), corr_result, 
    title='Serial Correlation', floatfmt='.4f'
)

Serial Correlation
Dependent variable: OLS residual, eᵢₜ

         Beta      Se    t-values
-----  ------  ------  ----------
eᵢₜ₋₁  0.5316  0.0123     43.2811
R² = 0.279
σ² = 0.011


In [267]:
#################### TEST FOR FE.3 ######################
#########################################################